In [1]:
import json
# asins_small.json to be used for meta data matching
json_file_asins = './data/asins_small.json'
# json_file_asins = './data/meta_asins.json'
with open(json_file_asins, "r") as file:
    asin_dict = json.load(file)
    
len(asin_dict) # 748224 for the full meta bag # 434236 after filtering out the ones with no title

29205

In [2]:
print("test")
asin_dict['B005TZGJBA'][0]

test


'The Spitfire Grill'

In [3]:
# read eval dataset from local
from datasets import load_from_disk

dataset_eval = load_from_disk("./data/gemma_chat_eval")
print("Loaded eval dataset from {}".format("./data/gemma_chat_eval"))

# dataset_eval.num_rows
# index_i = 6
index_i = 46
dataset_eval[index_i]['input'], \
dataset_eval[index_i]['output']

Loaded eval dataset from ./data/gemma_chat_eval


('Below is the previous historical purchases and reviews of the user:\n```\nItem title: Runaway Jury (Widescreen Edition) \n Item description: Product Description Set in New Orleans, this is the story of a mysterious man, Nick Easter who gets himself on the jury of a landmark case against a gun manufacturer in an attempt to influence the other jury members to vote a certain way. Direction: Gary Fleder Actors: Dustin Hoffman, Gene Hackman, Jennifer Beals. Special Features: Director Commentary; "Exploring the Scene" - Hackman & Hoffman\'s first scene together; "Off the Cuff" - Lead Actor Commentary by Dustin Hoffman & Gene Hackman; Five Featurettes - Making Of, Acting, Cinematography, Production, Editing. 2.35:1 format. Language: English / Add. French, Spanish / Sub. English, Spanish Year: 2003 Runtime: 122 minutes. Amazon.com Based on the bestseller by John Grisham, Runaway Jury is a slick thriller that\'s exciting enough to overcome the gaps in its plot. The ultimate target has been ch

In [4]:
from transformers import TextStreamer, GenerationConfig
from unsloth import FastLanguageModel
import torch

max_seq_length = 4096 # 8192 | Choose any! We auto support RoPE Scaling internally!
dtype = None # None for auto detection. Float16 for Tesla T4, V100, Bfloat16 for Ampere+
load_in_4bit = True # Use 4bit quantization to reduce memory usage. Can be False.
    
model, tokenizer = FastLanguageModel.from_pretrained(
    # model_name = "outputs/model_04242024_090830/", # YOUR MODEL YOU USED FOR TRAINING
    # model_name = "unsloth/gemma-2b-it-bnb-4bit",
    model_name = "outputs/checkpoint-1000",
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
)
FastLanguageModel.for_inference(model) # Enable native 2x faster inference
        
prompt_template = "<start_of_turn>user\n{}<end_of_turn>\n<start_of_turn>model\n{}"

input = dataset_eval[index_i]['input']
if input.endswith("title and description."):
    input = "".join(dataset_eval[index_i]['input'].split("\n")[:-1]) + "Please infer the user's preference based on historical purchases and reviews along with the user's intention, and then recommend an item for this user. Please just give the title of the recommended item."

# input = dataset_eval[index_i]['input'] + "\n Please also explain yourself in one sentence or two, why the user might be interested in your recommended item."
# input = "Hello, I am looking for a movie that will make me feel happy and excited. I love action movies with a lot of suspense and thrill. I also enjoy movies with a lot of drama and romance. Can you recommend a movie that will make me feel happy and excited? And please explain why you recommend this movie."
# input= "Below is the previous historical purchases and reviews of the user:\n```\nItem title: One Step Behind \n Item description: A dangerous man awakes in the care of a mysterious woman. Once the pieces of his past fall into place, he's faced with the stark choice of accepting his new found love or becoming the man he once was. \n rating: 4.0 \n review: Nice Hard Boiled Indie! Plenty of Bad Guys, Femme Fatales and one Big Bad Ass Reluctant Hero! If you like old school Film Noir -  more specifically Hard Boiled - you'll enjoy this little indie film. It reminded me a lot of the Coen Brother's BLOOD SIMPLE and another desert noir from the 90s starring Nick Cage RED ROCK WEST. It has that same pace and flavor to it and it pays of at the end. Well done!-------\nItem title: Untouched \n Item description: This legal drama explores the life-changing effects a secret abortion has on Mitch Thomas. A town startled by an infant found murdered in a dumpster, stirs Mitch, the reluctant attorney, conflicted by the love he lost, to unveil his haunting secret. \n rating: 4.0 \n review: Nice courtroom drama with it's heart in the right place. Beautifully shot and written, this is an intriguing story with a bittersweet ending. Really enjoyed it!-------\nItem title: The Man In The Silo \n Item description: A successful business man wakes up to find himself in a dilapidated grain silo, he must reconnect the dots of his traumatized memory to discover the truth of how he ended up there. \n rating: 5.0 \n review: This is a beautiful film full of great cinematic storytelling plus a classic re-hash of the Bernard Herman's Vertigo Score. Being a Hitchcock fan, I can honestly say that this is a true HITCHCOCKIAN film - which we hear a lot but hardly ever see. You can tell the film was made with so much attention to detail and passion by the great cinematography, transitions, acting, pace, acting and editing - and everything works together to achieve a great visual art piece. But don't get me wrong, it is also super exciting and entertaining as the film keeps you guessing from beginning to end. It is full of intrigue and suspense - and it is haunting. I highly recommend THE MAN IN THE SILO and I can almost guarantee you haven't seen anything like it!-------\n```\nAnd here is the user's intention: I really enjoy indie films with strong storytelling and emotional depth, so I'm looking for a movie that will steal my heart and make me want to fall in love all over again.\nPlease infer the user's preference based on historical purchases and reviews along with the user's intention, and then recommend an item for this user and provide its description."
# input = "hello, can you recommend me a random movie? Maybe a romantic one? And please describe the reason why you recommend this movie."

inputs = tokenizer(
[
    # "### Input:\n{inputs}\n\n### Response:{outputs}".format(inputs= input, outputs= ""),
    # "{inputs}".format(inputs= input),
    prompt_template.format(input, ""),    
], return_tensors = "pt").to("cuda")

num_beams_parameter = 5
custom_generation_config = GenerationConfig(
    bos_token_id=tokenizer.bos_token_id,
    eos_token_id=tokenizer.eos_token_id,
    pad_token_id=tokenizer.pad_token_id,
    asin_dict=asin_dict,
    tokenizer=tokenizer,
    return_dict_in_generate=True,
    output_scores=True,
    # output_logits=True,
    # do_sample=True,
    early_stopping=True,
    num_beams=num_beams_parameter, 
    num_return_sequences=num_beams_parameter,
    max_new_tokens=35,
    use_cache=True,
    temperature=1,
    num_beam_groups=5, # In this generation mode, `num_beams` should be divisible by `num_beam_groups`. `diversity_penalty` is not 0.0 or `num_beam_groups` is not 1, triggering group beam search. 
    diversity_penalty=0.9, # `diversity_penalty` should be greater than `0.0`, otherwise your groups will be identical.
)

# inputs = tokenizer(
# [
#     prompt_template.format(input, ""),    
# ], return_tensors = "pt").to("cuda")

outputs = model.generate(**inputs, generation_config=custom_generation_config)

# check CUDA memory usage
used_memory = round(torch.cuda.max_memory_allocated() / 1024 / 1024 / 1024, 3)
# if used_memory close to 14GB, empty the cache
if used_memory > 1:
    torch.cuda.empty_cache()
    print("CUDA memory usage is high. Cleared the cache.")
        
# print(outputs)
# print(tokenizer.batch_decode(outputs['sequences'], skip_special_tokens=True))
# outputs['scores'] is the tuple of probabilty distributions at every time step.

# text_streamer = TextStreamer(tokenizer)
# _ = model.generate(**inputs, streamer = text_streamer, max_new_tokens = 2048, num_beams=1, do_sample=True, generation_config=custom_generation_config, use_cache=True)

==((====))==  Unsloth: Fast Gemma patching release 2024.4
   \\   /|    GPU: Tesla T4. Max memory: 14.581 GB. Platform = Linux.
O^O/ \_/ \    Pytorch: 2.2.2. CUDA = 7.5. CUDA Toolkit = 12.1.
\        /    Bfloat16 = FALSE. Xformers = 0.0.25.post1. FA = False.
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth


Unused kwargs: ['_load_in_4bit', '_load_in_8bit', 'quant_method']. These kwargs are not used in <class 'transformers.utils.quantization_config.BitsAndBytesConfig'>.
Gemma's activation function should be approximate GeLU and not exact GeLU.
Changing the activation function to `gelu_pytorch_tanh`.if you want to use the legacy `gelu`, edit the `model.config` to set `hidden_activation=gelu`   instead of `hidden_act`. See https://github.com/huggingface/transformers/pull/29402 for more details.
Unsloth 2024.4 patched 18 layers with 18 QKV layers, 18 O layers and 18 MLP layers.
Using customized constraint logits processor


Creating trie...
new_token_input_ids_list[0]: [2]
new_token_input_ids_list[0]: [2]
new_token_input_ids_list[0]: [2]
new_token_input_ids_list[0]: [2]
new_token_input_ids_list[0]: [2]
new_token_input_ids_list[0]: [2, 235288]
new_token_input_ids_list[0]: [2, 651]
new_token_input_ids_list[0]: [2, 114167]
new_token_input_ids_list[0]: [2, 235288]
new_token_input_ids_list[0]: [2, 651]
new_token_input_ids_list[0]: [2, 235288, 781]
new_token_input_ids_list[0]: [2, 651, 10391]
new_token_input_ids_list[0]: [2, 114167, 13548]
new_token_input_ids_list[0]: [2, 235288, 781]
new_token_input_ids_list[0]: [2, 651, 10391]
new_token_input_ids_list[0]: [2, 235288, 781, 11523]
new_token_input_ids_list[0]: [2, 651, 10391, 110911]
new_token_input_ids_list[0]: [2, 114167, 13548, 10116]
new_token_input_ids_list[0]: [2, 235288, 781, 11523]
new_token_input_ids_list[0]: [2, 651, 10391, 110911]
new_token_input_ids_list[0]: [2, 235288, 781, 11523, 35124]
new_token_input_ids_list[0]: [2, 651, 10391, 110911, 591]
new_

In [6]:
# for k, items in outputs.items():
#     print(k)
# outputs['sequences'].size(), \
#     # outputs['sequences']

In [5]:
for i in range(len(outputs['sequences'])):
    # print(tokenizer.batch_decode(outputs['sequences'][i], skip_special_tokens=True))
    sequence = "".join(tokenizer.batch_decode(outputs['sequences'][i], skip_special_tokens=True))
    sequence = sequence.split("model\n")[1]
    print(sequence)
    if num_beams_parameter == 1: print(outputs['scores'][-1][0][1]) # this is for num_beam = 1
    else: print(outputs['sequences_scores'][i])

Crouching Tiger, Hidden Dragon
tensor(-0.2512, device='cuda:0')
The Last Samurai (Two-Disc Special Edition)
tensor(-0.6932, device='cuda:0')
Crouching Tiger, Hidden Dragon
tensor(-1.0386, device='cuda:0')
Kung Fu Master [DVD]
tensor(-1.2536, device='cuda:0')
The Last Samurai (Two-Disc Special Edition)
tensor(-1.5116, device='cuda:0')


In [7]:
import torch
from torch.eval.metrics import ReciprocalRank

metric = ReciprocalRank(k=1)
metric.update(torch.tensor([[0, 0.1, 0.6, 0.3], [0, 0, 1, 0]]), torch.tensor([2, 1]))

ModuleNotFoundError: No module named 'torch.eval'

In [ ]:
import math
# outputs['scores'][-1]
# check every token's probability distribution, if there is a token with a positive probability
for i, token in enumerate(outputs['scores'][-1][0]):
    if token != -math.inf:
        print(f"index: {i}, Score: {token}")
        print(tokenizer.decode(i))
        print("\n")

In [ ]:
# read eval dataset from local
from datasets import load_from_disk

dataset_eval = load_from_disk("./data/gemma_chat_eval")
print("Loaded eval dataset from {}".format("./data/gemma_chat_eval"))

index_i = 16
dataset_eval[index_i]['input'], \
dataset_eval[index_i]['output']

In [ ]:
input_ids = [235285, 235303,    529,   2065,  64802,   1646]
result = tokenizer.decode(input_ids)
print(result)